In [107]:
import matplotlib.pyplot as plt
import os
import tensorflow as tf
import matplotlib
import keras.backend as K
matplotlib.style.use('ggplot')

In [108]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = 'training/'
VALID_DATA_DIR = 'validation/'

In [109]:
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)
train_generator = datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    shuffle=True,
    target_size=IMAGE_SHAPE,
)
valid_generator = datagen.flow_from_directory(
    VALID_DATA_DIR,
    shuffle=False,
    target_size=IMAGE_SHAPE,
)

Found 216 images belonging to 2 classes.
Found 54 images belonging to 2 classes.


In [110]:
def build_model(num_classes):
    model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu', 
                           input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
    ])
    return model
model = build_model(num_classes=2)

In [111]:
def get_f1(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    
    return f1_val

In [112]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=[get_f1]
)
print(model.summary())

Model: "sequential_15"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_39 (Conv2D)          (None, 222, 222, 16)      448       
                                                                 
 max_pooling2d_36 (MaxPoolin  (None, 111, 111, 16)     0         
 g2D)                                                            
                                                                 
 conv2d_40 (Conv2D)          (None, 109, 109, 16)      2320      
                                                                 
 max_pooling2d_37 (MaxPoolin  (None, 54, 54, 16)       0         
 g2D)                                                            
                                                                 
 conv2d_41 (Conv2D)          (None, 52, 52, 32)        4640      
                                                                 
 max_pooling2d_38 (MaxPoolin  (None, 26, 26, 32)     

In [113]:
EPOCHS = 2
BATCH_SIZE = 16
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // BATCH_SIZE // 2,
                    epochs=EPOCHS,
                    validation_data=valid_generator,
                    validation_steps= valid_generator.samples // BATCH_SIZE // 2,
                    verbose=1
                    )

Epoch 1/2
6/6 [==============================] - 9s 1s/step - loss: 0.7139 - get_f1: 0.5955 - val_loss: 0.3067 - val_get_f1: 0.8437
Epoch 2/2
6/6 [==============================] - 5s 806ms/step - loss: 0.5922 - get_f1: 0.6806 - val_loss: 0.5302 - val_get_f1: 0.7500


In [114]:
model.save_weights('firstname_secondname')